![Degirum banner](https://raw.githubusercontent.com/DeGirum/PySDKExamples/main/images/degirum_banner.png)
## AI Inference on image dataset retrieved from a cloud
This notebook is an example how to use DeGirum PySDK to do AI inference of an image dataset with evaluation
of precision/recall metrics. Image dataset is retrieved from the cloud using `fiftyone` API.

This script works with the following inference options:

1. Run inference on DeGirum Cloud Platform;
2. Run inference on DeGirum AI Server deployed on a localhost or on some computer in your LAN or VPN;
3. Run inference on DeGirum ORCA accelerator directly installed on your computer.

To try different options, you need to specify the appropriate `hw_location` option. 

You also need to specify your cloud API access token in `degirum_cloud_token`.

In [ ]:
# make sure degirum-tools package is installed
!pip show degirum-tools || pip install degirum-tools
!pip show fiftyone || pip install fiftyone

#### Specify where you want to run your inferences and dataset parameters here

In [ ]:
# hw_location: where you want to run inference
#     "@cloud" to use DeGirum cloud
#     "@local" to run on local machine
#     IP address for AI server inference
# model_zoo_url: url/path for model zoo
#     cloud_zoo_url: valid for @cloud, @local, and ai server inference options
#     '': ai server serving models from local folder
#     path to json file: single model zoo in case of @local inference
# foz_dataset_name: name of desired dataset to retrieve; 
#     see https://voxel51.com/docs/fiftyone/user_guide/dataset_zoo/datasets.html
# foz_classes: list of class labels to retrieve; None for all classes
# foz_splits: which splits to download ("train", "validation", "test")
# samples_num: number of samples to retrieve from dataset
# model_name: name of model to be used for inference
hw_location = "@cloud"
model_zoo_url = "https://cs.degirum.com/degirum/public"
foz_dataset_name = "coco-2017"
foz_classes = None # ["car", "cup", "person"]
foz_splits = "validation"
samples_num = 1000
model_name = "yolo_v5s_coco--512x512_quant_n2x_orca1_1"

#### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg, degirum_tools

# connect to AI inference engine
zoo = dg.connect(hw_location, model_zoo_url, degirum_tools.get_token())

In [ ]:
import sys, IPython, fiftyone as fo

# download dataset
dataset = fo.zoo.load_zoo_dataset(
    foz_dataset_name,
    dataset_dir="./temp/my-dataset", 
    classes=foz_classes,
    split=foz_splits,
    max_samples=samples_num,
    shuffle=True,
    drop_existing_dataset=False if 'is_first_run' in globals() else True)
print(dataset)
is_first_run = False

print("Extracting dataset samples...")
all_samples = dataset.head(dataset.count()) # retrieve all dataset samples
all_files = [s.filepath for s in all_samples]
print("...done")

In [ ]:
# load model and set low confidence threshold for proper statistics calculation
model = zoo.load_model(model_name, output_confidence_threshold = 0.1)
 
print("Running inference:")
progress = degirum_tools.Progress(len(all_files))
for n, res in enumerate(model.predict_batch(all_files)):
    if model.image_backend == 'pil':
        w, h = res.image.size
    else: # opencv
        w = res.image.shape[1]
        h = res.image.shape[0]
    
    detections = []
    for box in res.results:
        # Convert to [top-left-x, top-left-y, width, height]
        # in relative coordinates in [0, 1] x [0, 1]
        x1, y1, x2, y2 = box["bbox"]
        rel_box = [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h]
        detections.append(fo.Detection(label=box["label"], bounding_box=rel_box, confidence=box["score"]))
    all_samples[n]["predictions"] = fo.Detections(detections=detections)
    all_samples[n].save()
    progress.step()

In [ ]:
# run evaluation on predictions
eval_result = dataset.evaluate_detections("predictions", classes=foz_classes, compute_mAP=True)

# print some evaluation results
print( f"mAP = {eval_result.mAP():.2f}\n")
eval_result.print_report(classes=foz_classes)